In [ ]:
import pandas as pd
import numpy as np
import joblib
from lightgbm import LGBMRegressor
import warnings
warnings.filterwarnings("ignore")

# Try importing the Mitsui server (Kaggle-only)
try:
    import mitsui
except:
    mitsui = None


In [ ]:
# Load the trained model
model = joblib.load("../models/lgbm_tuned_model.pkl")


In [ ]:
def predict(market_features_df: pd.DataFrame, revealed_targets_df: pd.DataFrame) -> pd.DataFrame:
    df = market_features_df.copy()
    
    # --- Feature Engineering (same logic as training) ---
    for lag in [1, 2, 3]:
        for col in ['LME_CA_Close', 'LME_ZS_Close', 'target_2']:
            df[f'{col}_lag{lag}'] = df[col].shift(lag)

    for window in [3, 7, 14]:
        df[f'CA_roll_mean_{window}'] = df['LME_CA_Close'].rolling(window).mean()
        df[f'ZS_roll_mean_{window}'] = df['LME_ZS_Close'].rolling(window).mean()
        df[f'target_roll_std_{window}'] = df['target_2'].rolling(window).std()

    df.dropna(inplace=True)
    
    # Predict the last available row
    features = df.iloc[[-1]]  # last row only
    pred_val = model.predict(features)[0]
    
    # Fill all 424 targets with zeros, then insert our prediction into target_2 (index 1)
    out = pd.DataFrame([[0.0]*424], columns=[f"target_{i}" for i in range(424)])
    out["target_2"] = pred_val
    
    return out


In [ ]:
if mitsui:
    mitsui.run(predict)
else:
    print("Not running on Kaggle. You can test locally using mitsui_demo_submission.py")
